# Import libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
df.head()

### Counts quality values

In [ ]:
df['quality'].value_counts()

# Exploratory Data Analysis
## See relation between each feature to quality

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(25,5))
sns.boxenplot('quality', 'fixed acidity', data=df, ax=ax1)
sns.boxenplot('quality', 'volatile acidity', data=df, ax=ax2)
sns.boxenplot('quality', 'citric acid', data=df, ax=ax3)
sns.boxenplot('quality', 'residual sugar', data=df, ax=ax4)
plt.show()

**From above graph, we can see that lower volatile acidity correlate to higher quality, while citric acid had positive correlation to quality. but we can see clear pattern in fixed acidity and residual sugar**

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(25,5))
sns.boxenplot('quality', 'chlorides', data=df, ax=ax1)
sns.boxenplot('quality', 'free sulfur dioxide', data=df, ax=ax2)
sns.boxenplot('quality', 'total sulfur dioxide', data=df, ax=ax3)
sns.boxenplot('quality', 'density', data=df, ax=ax4)
plt.show()

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(25,5))
sns.boxenplot('quality', 'pH', data=df, ax=ax1)
sns.boxenplot('quality', 'sulphates', data=df, ax=ax2)
sns.boxenplot('quality', 'alcohol', data=df, ax=ax3)
plt.show()

## Create new Dataframe
### Change quality into 2 class : Nice and Bad

In [ ]:
new_df = df

new_df['quality'] = np.where(new_df['quality']>=7, 'Nice', 'Bad')

In [ ]:
new_df.head()

In [ ]:
new_df['quality'].value_counts()

**This data is not evenly distributed for 2 types of classes**

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(25,5))

sns.kdeplot('fixed acidity', data=new_df, hue='quality', ax=ax1)
sns.kdeplot('volatile acidity', data=new_df, hue='quality', ax=ax2)
sns.kdeplot('citric acid', data=new_df, hue='quality', ax=ax3)
sns.kdeplot('residual sugar', data=new_df, hue='quality', ax=ax4)
# sns.kdeplot('chlorides', data=new_df, hue='quality', ax=ax5)

**we can't get insight from distribution plot, since the data didn't distributed equally**

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(25,5))
sns.boxenplot('quality', 'fixed acidity', data=new_df, ax=ax1)
sns.boxenplot('quality', 'volatile acidity', data=new_df, ax=ax2)
sns.boxenplot('quality', 'citric acid', data=new_df, ax=ax3)
sns.boxenplot('quality', 'residual sugar', data=new_df, ax=ax4)
plt.show()

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(25,5))
sns.boxenplot('quality', 'chlorides', data=new_df, ax=ax1)
sns.boxenplot('quality', 'free sulfur dioxide', data=new_df, ax=ax2)
sns.boxenplot('quality', 'total sulfur dioxide', data=new_df, ax=ax3)
sns.boxenplot('quality', 'density', data=new_df, ax=ax4)
plt.show()

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(25,5))
sns.boxenplot('quality', 'pH', data=new_df, ax=ax1)
sns.boxenplot('quality', 'sulphates', data=new_df, ax=ax2)
sns.boxenplot('quality', 'alcohol', data=new_df, ax=ax3)
plt.show()

# Training Models

#### Plot loss function

In [ ]:
def plot_loss():
    plt.figure(figsize=(10,5), dpi=80)

    history_df = pd.DataFrame(history.history)

    plt.plot(np.arange(0,history_df.shape[0]), history_df['loss'])
    plt.plot(np.arange(0,history_df.shape[0]), history_df['val_loss'])
    plt.show()
    print("\nMinimum validation loss: {}".format(history_df['val_loss'].min()))

# Model 1 : Neural Network Regression models to predict 5 class (quality : 3,4,5,6,7,8)

In [ ]:
df = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

X = df.drop(['quality'], axis=1)
X = normalize(X)

y = df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=50)

# Regression

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, input_shape=[11], activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='relu')
])

model.compile(loss='mean_squared_error',
             optimizer=tf.keras.optimizers.SGD(lr=0.0001))

checkpointer = ModelCheckpoint(filepath="model_1.h5", verbose=1, save_best_only=True)

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    steps_per_epoch=100,
                    epochs=50,
                    callbacks=[checkpointer])


model = "./model_1.h5"
model = tf.keras.models.load_model(model, compile = True)
prediction_train = np.round(model.predict(X_train))
prediction_val = np.round(model.predict(X_test))

print('\nTrain Accuracy : {}' .format(metrics.accuracy_score(prediction_train, y_train)))
print('Validation Accuracy : {}' .format(metrics.accuracy_score(prediction_val, y_test)))

plot_loss()

# Model 2 : Neural Network Classification model to predict 5 class (quality : 3,4,5,6,7,8)

In [ ]:
df = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

X = df.drop(['quality'], axis=1)
X = normalize(X)

y = df['quality']
y = np.array(tf.one_hot(y, 6))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=50)

# softmax classification

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, input_shape=[11], activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(loss='CategoricalCrossentropy',
             optimizer=tf.keras.optimizers.SGD(lr=0.0001))

checkpointer = ModelCheckpoint(filepath="./model_2.h5", verbose=1, save_best_only=True)

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    steps_per_epoch=100,
                    epochs=500,
                    callbacks=[checkpointer])


model = "./model_2.h5"
model = tf.keras.models.load_model(model, compile = True)
prediction_train = np.round(model.predict(X_train))
prediction_val = np.round(model.predict(X_test))

print('\nTrain Accuracy : {}' .format(metrics.accuracy_score(prediction_train, y_train)))
print('Validation Accuracy : {}' .format(metrics.accuracy_score(prediction_val, y_test)))

# Model 3 : Neural Network for Binary classification model (Nice or Bad)

In [ ]:
df = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
new_df = df
new_df['quality'] = np.where(new_df['quality']>=7, 1, 0)


X = new_df.drop(['quality'], axis=1)
X = normalize(X)

y = new_df['quality']
# y = np.array(tf.one_hot(y, 2))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=50)

# softmax classification

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, input_shape=[11], activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='BinaryCrossentropy',
             optimizer=tf.keras.optimizers.SGD(lr=0.0001),
             metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath="./model_3.h5", verbose=1, save_best_only=True)

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    steps_per_epoch=100,
                    epochs=500,
                    callbacks=[checkpointer])


model = "./model_3.h5"
model = tf.keras.models.load_model(model, compile = True)
prediction_train = np.round(model.predict(X_train))
prediction_val = np.round(model.predict(X_test))

print('\nTrain Accuracy : {}' .format(metrics.accuracy_score(prediction_train, y_train)))
print('Validation Accuracy : {}' .format(metrics.accuracy_score(prediction_val, y_test)))

plot_loss()

# Traditional Machine Learning Technique

In [ ]:
df = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
new_df = df
new_df['quality'] = np.where(new_df['quality']>=7, 1, 0)


X = new_df.drop(['quality'], axis=1)
X = normalize(X)

y = new_df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=50)


model = [SGDClassifier(),
        DecisionTreeClassifier(),
        LinearSVC(),
        GaussianNB(),
        KNeighborsClassifier(),
        RandomForestClassifier(),
        LogisticRegression(),
        GradientBoostingClassifier()
        ]

for models in model:
    model = models
    model.fit(X_train,y_train)
    prediction_train = model.predict(X_train)
    prediction_val = model.predict(X_test)

    print('\n{} Train Accuracy : {}' .format(model, metrics.accuracy_score(prediction_train, y_train)))
    print('{} Validation Accuracy : {}' .format(model, metrics.accuracy_score(prediction_val, y_test)))